# Step 1. 데이터 다운로드

In [134]:
import glob
import os, re
import tensorflow as tf

In [135]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
# Colab용 
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2022-12-30 08:37:35--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.250.4.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2022-12-30 08:37:35 (124 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]




# Step 2. 데이터 읽어오기

In [137]:
txt_file_path = "/content/drive/MyDrive/lyric_data/*" #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장
txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당
raw_corpus = [] 
# C:\Users\Soonju\Desktop\Aiffel\Ex6_Lyricist
# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='UTF8') as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
# print("Examples:\n", raw_corpus[:3])
print(raw_corpus[:10])

데이터 크기: 187088
["I'mma start it from the bottom", "I'll show you how to flip a dollar", 'I got food in my dining room', "I'm better, I'm better, I'm better", "It's another day, another chance", 'I wake up, I wanna dance', 'So as long as I got my friends', "I'm better, I'm better, I'm better He say I'm hot, I'm so fuego", 'Pull up on him in my vehicle', "He say I'm pretty, I'm pretty"]


# Step 3. 데이터 정제



In [138]:
corpus = []

for sentence in raw_corpus:
    if len(sentence.split()) > 15: continue
    
    def preprocess_sentence(sentence):
        sentence = sentence.lower().strip() # 1
        sentence = re.sub(r"([?.!,¿,'])", r" \1 ", sentence) # 2
        sentence = re.sub(r'[" "]+', " ", sentence) # 3
        sentence = re.sub(r"[^a-zA-Z?.!,¿,']+", " ", sentence) # 4
        sentence = sentence.strip() # 5
        sentence = '<start> ' + sentence + ' <end>' # 6 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
        return sentence
    preprocessed_sentence = preprocess_sentence(sentence)    
    corpus.append(preprocessed_sentence)
corpus[:10]

["<start> i ' mma start it from the bottom <end>",
 "<start> i ' ll show you how to flip a dollar <end>",
 '<start> i got food in my dining room <end>',
 "<start> i ' m better , i ' m better , i ' m better <end>",
 "<start> it ' s another day , another chance <end>",
 '<start> i wake up , i wanna dance <end>',
 '<start> so as long as i got my friends <end>',
 "<start> i ' m better , i ' m better , i ' m better he say i ' m hot , i ' m so fuego <end>",
 '<start> pull up on him in my vehicle <end>',
 "<start> he say i ' m pretty , i ' m pretty <end>"]

# Step 4. 평가 데이터셋 분리

## 1. 토큰화 

1.   text 분리하여 인덱스부여
2.   텐서화



In [139]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus) # 1단계 :corpus의 text를 list로 변환

#     word_index = tokenizer.word_index  # 번외: word 별로 index를 붙여 단어 하나당 인덱스 하나씩 부여한 딕셔너리.
#     print(word_index) 
   
    tensor = tokenizer.texts_to_sequences(corpus) # 2단계 :text를 sequence,tensor로 변환   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) # 끝에 pad주기 
    print(tensor,tokenizer)
    return tensor, tokenizer


tensor, tokenizer = tokenize(corpus)


[[  2   6   4 ...   0   0   0]
 [  2   6   4 ...   0   0   0]
 [  2   6  41 ...   0   0   0]
 ...
 [  2   6   4 ...   0   0   0]
 [  2 113 664 ...   0   0   0]
 [  2   9  51 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x7f13fa5bc4c0>


## 2. train, target 분리하기

In [140]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    
print(src_input[0])
print(tgt_input[0])

[  2   6   4 530 342  12  75   7 876   3   0   0   0   0]
[  6   4 530 342  12  75   7 876   3   0   0   0   0   0]


In [141]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256                                # 한번에 가져올 문장 수 
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   

Tdata = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # 주어진 데이터 sequences(src_input)와 label(tgt_input)을 묶어서 조각으로 만들고 같이 사용
Tdata = Tdata.shuffle(BUFFER_SIZE)
Tdata = Tdata.batch(BATCH_SIZE, drop_remainder=True)
Tdata


<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [142]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, test_size= 0.2, random_state=1234)

print(enc_train.shape,enc_val.shape,dec_train.shape,dec_val.shape)



(143753, 14) (35939, 14) (143753, 14) (35939, 14)


# Step 5. 인공지능 만들기

## 1. 모델만들기

In [143]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.


optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(Tdata, epochs=10)

# split 제대로 하지 않았을때, Epoch 10/10
# 78/78 [==============================] - 17s 212ms/step - loss: 0.4697

# split 제대로 되었을 때, Epoch 10/10
# 701/701 [==============================] - 117s 167ms/step - loss: 1.9620

Epoch 1/10
701/701 [==============================] - 122s 167ms/step - loss: 3.3427
Epoch 2/10
701/701 [==============================] - 117s 167ms/step - loss: 2.8694
Epoch 3/10
701/701 [==============================] - 117s 166ms/step - loss: 2.6895
Epoch 4/10
701/701 [==============================] - 117s 166ms/step - loss: 2.5535
Epoch 5/10
701/701 [==============================] - 117s 167ms/step - loss: 2.4349
Epoch 6/10
701/701 [==============================] - 117s 166ms/step - loss: 2.3272
Epoch 7/10
701/701 [==============================] - 117s 167ms/step - loss: 2.2267
Epoch 8/10
701/701 [==============================] - 117s 167ms/step - loss: 2.1335
Epoch 9/10
701/701 [==============================] - 117s 167ms/step - loss: 2.0452
Epoch 10/10
701/701 [==============================] - 117s 167ms/step - loss: 1.9620


## 2. 작문 함수 만들기

In [144]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

type(sentence)

str

## 3. 작문 시키기

In [145]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you , i love you <end> '

In [146]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256                                # 한번에 가져올 문장 수 
steps_per_epoch = len(enc_val) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   

data = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)) # 주어진 데이터 sequences(src_input)와 label(tgt_input)을 묶어서 조각으로 만들고 같이 사용
data = data.shuffle(BUFFER_SIZE)
data = data.batch(BATCH_SIZE, drop_remainder=True)
data
loss = model.evaluate(data)
print('val loss',loss) # val loss 1.8701671361923218


140/140 [==============================] - 10s 65ms/step - loss: 1.8702
val loss 1.8701671361923218


회고

NLP의 전처리 과정이 상당히 복잡하고 어려웠다.
예전 노드에서 익숙치 않았던 데이터 불러오는 법, re 정규표현식, 텍스트를 다루던 모든 것에 대해 다시 복습해야 했다.
더불어 벡터와 텐서에 대해서도 다시 공부할 수 밖에 없었다.
컴퓨터의 모국어는 숫자인데 내가 사람의 언어를 컴퓨터에게 전달해주는데에 모자람을 느꼈다. 통역이 안되는 느낌..
특히 정규표현식을 다시 복습할 때에 제일 많이 느낀 것 같다.

컴퓨터는 '숫자'와 '차원'으로 표현한다.  
그 작업을 '데이터 전처리'에서도 하고, '모델링'에서도 한다.  
데이터 전처리에서는 '토큰화'를 통해서,  
모델링에서는 'embedding layer'가 그러하다.

문자를 숫자로 바꾸고,  
이를 텐서로 바꿔 lookup table (aka.단어장)을 만들어놔야만  
내가 이후에 컴퓨터에게 일을 시킬 수 있단 사실 잊지않기.  
컴퓨터와 소통하는 절차에 대해 생각해보게 되었다.

**최종결과**
val loss 1.8701671361923218